In [1]:
import polars as pl
import json

In [9]:
with open('./raw-data/1.json', 'r') as f:
    # Lee todo el contenido del archivo y lo guarda en una cadena
    texto = f.read()

In [10]:
lineas = texto.split('\n')


In [11]:
lineas[0]

'{"name": "Porter Pharmacy", "address": "Porter Pharmacy, 129 N Second St, Cochran, GA 31014", "gmap_id": "0x88f16e41928ff687:0x883dad4fd048e8f8", "description": null, "latitude": 32.3883, "longitude": -83.3571, "category": ["Pharmacy"], "avg_rating": 4.9, "num_of_reviews": 16, "price": null, "hours": [["Friday", "8AM\\u20136PM"], ["Saturday", "8AM\\u201312PM"], ["Sunday", "Closed"], ["Monday", "8AM\\u20136PM"], ["Tuesday", "8AM\\u20136PM"], ["Wednesday", "8AM\\u201312PM"], ["Thursday", "8AM\\u20136PM"]], "MISC": {"Service options": ["In-store shopping", "Same-day delivery"], "Health & safety": ["Mask required", "Staff required to disinfect surfaces between visits"], "Accessibility": ["Wheelchair accessible entrance"], "Planning": ["Quick visit"]}, "state": "Open \\u22c5 Closes 6PM", "relative_results": ["0x88f16e41929435cf:0x5b2532a2885e9ef6", "0x88f16c32716531c1:0x5f19bdaa5044e4fa", "0x88f16e6e3f4a21df:0xcf495da9bb4d89ea"], "url": "https://www.google.com/maps/place//data=!4m2!3m1!1s0

In [11]:
names = []
gmap_id  = []
descrptions  = []
latitudes  = []
longitudes  = []
categories  = []
avg_ratings  = []
num_of_reviews   = []
prices = []
MISCs  = []
urls  = []
for linea in lineas:
    if linea.strip():  # Ignora líneas en blanco
        objeto_json = json.loads(linea)
        names.append(objeto_json['name'])
        gmap_id.append(objeto_json['gmap_id'])
        descrptions.append(objeto_json['description'])
        latitudes.append(objeto_json['latitude'])
        longitudes.append(objeto_json['longitude'])
        if objeto_json['category']:
            c=0
            for categoria in objeto_json['category']:
                if 'hotel' in categoria.lower():
                    c+=1
                    categories.append(objeto_json['category'])
                    break
            if c == 0:
                categories.append(None)
        else:
            categories.append(None)
        avg_ratings.append(objeto_json['avg_rating'])
        num_of_reviews.append(objeto_json['num_of_reviews'])
        servicios = []
        if (objeto_json['MISC'] != 'null') and (objeto_json['MISC'] != 'None') and objeto_json['MISC']:    
            for key,value in objeto_json['MISC'].items():
                if (value != 'null') and (value != 'None') and value and key not in ['Health & safety', 'Planning']:
                    servicios.extend(value)
        else:
            servicios = None
        if servicios == []:
            servicios = None
        MISCs.append(servicios)
        urls.append(objeto_json['url'])

In [12]:
data = {
    "name": names,
    "gmap_id": gmap_id,
    "descrption": descrptions,
    "latitude": latitudes,
    "longitude": longitudes,
    "category": categories,
    "avg_rating": avg_ratings,
    "num_of_reviews": num_of_reviews,
    "facilities": MISCs,
    "url": urls
}

df = pl.DataFrame(data)

In [13]:
df

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
str,str,str,f64,f64,list[str],f64,i64,list[str],str
"""Porter Pharmac…","""0x88f16e41928f…",null,32.3883,-83.3571,null,4.9,16,"[""In-store shopping"", ""Same-day delivery"", ""Wheelchair accessible entrance""]","""https://www.go…"
"""City Textile""","""0x80c2c98c0e3c…",null,34.018891,-118.21529,null,4.5,6,null,"""https://www.go…"
"""San Soo Dang""","""0x80c2c778e3b7…",null,34.058092,-118.29213,null,4.4,18,"[""Takeout"", ""Dine-in"", … ""Casual""]","""https://www.go…"
"""Nova Fabrics""","""0x80c2c89923b2…",null,34.023669,-118.23293,null,3.3,6,"[""In-store shopping"", ""Checks"", … ""Credit cards""]","""https://www.go…"
"""Nobel Textile …","""0x80c2c632f933…",null,34.036694,-118.249421,null,4.3,7,"[""In-store pickup""]","""https://www.go…"
"""Matrix Interna…","""0x80c2cf163db6…",null,34.015505,-118.181839,null,3.5,6,"[""Wheelchair accessible entrance""]","""https://www.go…"
"""Vons Chicken""","""0x80dd2b4c8555…",null,33.916402,-118.010855,null,4.5,18,"[""Outdoor seating"", ""Curbside pickup"", … ""College students""]","""https://www.go…"
"""Polished Nail …","""0x87ec238afdd8…",null,41.614198,-93.847269,null,4.0,86,"[""Identifies as women-led"", ""LGBTQ friendly"", … ""Credit cards""]","""https://www.go…"
"""Sweet Rewards …","""0x87ec235c54d2…",null,41.616079,-93.865487,null,4.7,21,"[""Delivery""]","""https://www.go…"


In [14]:
df1 = df.filter(~pl.col('category').is_null())

In [15]:
df1

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
str,str,str,f64,f64,list[str],f64,i64,list[str],str
"""Basecamp Guest…","""0x54907fca2751…",null,47.471332,-121.686928,"[""Hotel""]",5.0,8,null,"""https://www.go…"
"""Sugar River Lo…","""0x88062b28cb53…",null,42.619862,-89.37789,"[""Hotel""]",4.7,8,null,"""https://www.go…"
"""Meraki Lodge""","""0x549b9d4ad8f1…",null,47.887652,-120.120376,"[""Hotel"", ""Group accommodation"", … ""Motel""]",4.9,15,null,"""https://www.go…"
"""View House at …","""0x549b9db4d736…",null,47.888291,-120.12412,"[""Hotel""]",4.4,8,null,"""https://www.go…"
"""Station Lakefr…","""0x4d55cee0af10…",null,45.99872,-89.490867,"[""Resort hotel""]",4.7,3,null,"""https://www.go…"
"""Kings Island R…","""0x8840f7fb0543…","""No-frills hote…",39.355061,-84.264094,"[""Hotel"", ""Conference center"", ""Resort hotel""]",2.8,28,null,"""https://www.go…"
"""Garden City, S…","""0x890040a32810…",null,33.592541,-79.009323,"[""Hotel""]",3.0,2,null,"""https://www.go…"
"""Garden City Ka…","""0x8709f120d869…",null,38.050028,-100.847887,"[""Hotel""]",4.3,8,null,"""https://www.go…"
"""Howard Johnson…","""0x88404e8cd7f7…","""Unfussy hotel …",39.293253,-84.487513,"[""Hotel"", ""Inn"", … ""Motel""]",1.8,18,null,"""https://www.go…"


In [16]:
df1.null_count()

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,625,0,0,0,0,0,706,0


In [8]:
df1.shape

(0, 10)

In [17]:
df1=df1.unique(subset=['gmap_id'], keep='first')

In [7]:
df1.shape

(0, 10)

In [18]:
df1.null_count()

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,568,0,0,0,0,0,644,0


In [19]:
df1.write_parquet('./clean-data/x.parquet')

In [99]:
df1.filter(~pl.col('facilities').is_null())

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
str,str,str,f64,f64,list[str],f64,i64,list[str],str
"""Friendship Isl…","""0x8800ba5679c8…",null,43.976601,-89.812065,"[""Cottage rental"", ""Boat rental service"", … ""Swimming lake""]",4.9,38,"[""Wheelchair accessible entrance""]","""https://www.go…"
"""Deer Acres""","""0x87ca54e82f3e…",null,35.194869,-94.424416,"[""Zoo"", ""Resort hotel""]",5.0,8,"[""Good for kids""]","""https://www.go…"
"""Reynolds Lake …","""0x88f6f2d3d3ce…",null,33.470406,-83.245727,"[""Golf course"", ""Resort hotel""]",5.0,5,"[""Wheelchair accessible entrance""]","""https://www.go…"
"""Papakea Resort…","""0x79552bbbf2c0…",null,20.950901,-156.688559,"[""Vacation home rental agency"", ""Resort hotel"", ""Tourist attraction""]",3.9,8,"[""Onsite services"", ""Online appointments""]","""https://www.go…"
"""AC Art & Mirro…","""0x89c2f7975db5…",null,40.82794,-74.011248,"[""Picture frame shop"", ""Glass & mirror shop"", … ""Print shop""]",5.0,1,"[""In-store shopping""]","""https://www.go…"
"""Chauncy's""","""0x89fb30e4da73…",null,41.653414,-70.280301,"[""Bar & grill"", ""American restaurant"", … ""Hotel""]",4.5,8,"[""Takeout"", ""Dine-in"", … ""Tourists""]","""https://www.go…"
"""North Myrtle B…","""0x890062d577d1…",null,33.807929,-78.701414,"[""Condominium rental agency"", ""Golf course"", ""Resort hotel""]",4.8,8,"[""Wheelchair accessible entrance""]","""https://www.go…"
"""Oceanfront Res…","""0x8833b69c357a…",null,30.25532,-85.958831,"[""Condominium rental agency"", ""Hotel"", … ""Vacation home rental agency""]",4.5,16,"[""Wheelchair accessible entrance""]","""https://www.go…"
"""The Home Depot…","""0x88690b88db53…","""Chain home imp…",38.183396,-85.680111,"[""Wholesaler"", ""Cleaning products supplier"", … ""Plumbing supply store""]",2.7,7,"[""Wheelchair accessible entrance""]","""https://www.go…"


In [20]:
dfx = pl.read_parquet('./clean-data/x.parquet')

In [21]:
dfx

name,gmap_id,descrption,latitude,longitude,category,avg_rating,num_of_reviews,facilities,url
str,str,str,f64,f64,list[str],f64,i64,list[str],str
"""Redwood Motel""","""0x87909c44aaf9…",null,41.452206,-97.725204,"[""Motel"", ""Hotel""]",4.1,17,null,"""https://www.go…"
"""Anchor STUDIOS…","""0x88d9ba0c6136…",null,25.830137,-80.28346,"[""Hotel"", ""Furnished apartment building""]",3.2,28,null,"""https://www.go…"
"""Deer Acres""","""0x87ca54e82f3e…",null,35.194869,-94.424416,"[""Zoo"", ""Resort hotel""]",5.0,8,"[""Good for kids""]","""https://www.go…"
"""Presbyterian C…","""0x8819f775faa2…",null,42.660516,-86.215367,"[""Resort hotel""]",5.0,3,null,"""https://www.go…"
"""Holiday Inn St…","""0x87c01b06036d…","""Contemporary l…",39.766152,-94.857296,"[""Hotel""]",2.8,8,null,"""https://www.go…"
"""Now Voyager Cr…","""0x880f7b6fe929…",null,42.41979,-88.616029,"[""Travel agency"", ""Airline"", … ""Hotel""]",5.0,2,null,"""https://www.go…"
"""Campbell Colle…","""0x862833975d49…",null,32.298545,-90.209289,"[""Hotel""]",4.5,2,null,"""https://www.go…"
"""Gables Inn""","""0x89e58d258567…","""Victorian-styl…",41.173612,-71.561202,"[""Hotel""]",4.8,8,null,"""https://www.go…"
"""Rio Masquerade…","""0x80c8c544668d…",null,36.116408,-115.186322,"[""Hotel""]",4.0,28,null,"""https://www.go…"
